In [22]:
import os
base_path = os.path.dirname(os.path.abspath(""))
base_path = base_path + "/headers/"
print(base_path)
header_path = base_path + 'header_data_treatment.ipynb'
%run $header_path

C:\Users\raphael.wargon\Documents\GitHub\sorting_univ\script/headers/
default
initalizing spark nlp session w gpu


In [2]:
int_save_path = save_path + 'int_data_lab\\'

In [3]:
int_save_path

'E:\\panel_fr_res\\int_data_lab\\'

# Load data

In [28]:
works_au_af = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'works_au_af.parquet')
               .filter( func.col('publication_year').between(1950,2020) )
               .select('work_id','publication_year','author_id','source_id','citations','primary_topic','inst_id','country')

              )

In [29]:
journals_ranking = (spark.read.format('parquet').load('file:\\' + main_path_openalex + 'journals_ranking_year.parquet')
                    .withColumn('source_id', func.regexp_replace(func.col('source_id'), 'https://openalex.org/', ''))
                   )
journals_ranking.show()

+-----------+----+-----------+--------------------+------------------+
|  source_id|year|source_type|     rank_source_pct|   abs_rank_source|
+-----------+----+-----------+--------------------+------------------+
| S100014387|2014|    journal|  0.9673077009503904| 407.3062015503876|
| S100016587|1997|    journal|  0.6246648793565683|            1379.0|
|S1000263748|2012|    journal|  0.8371196475285695|            2343.0|
| S100028386|1995|    journal|  0.7268862911795961|             256.0|
| S100045985|1972|    journal|  0.8776194467728415|             146.0|
| S100062655|2021|    journal|                 0.0|               1.0|
|S1000839972|2015|    journal|  0.8853067047075607|             401.0|
|S1000944648|2012|    journal|  0.4543330087633885|            2603.0|
| S100105777|1991|    journal|  0.5352664576802508|             549.0|
| S100105777|2010|    journal| 0.48626549156502025| 4091.909090909091|
|S1001192252|2011|    journal|0.030941856511390683|            5374.0|
|  S10

# Build the raw dataset

In [30]:
inst = spark.read.parquet('file:\\' + save_path + 'inst_fr.parquet')

In [7]:
au_inst_level_flow = (works_au_af
                      .join(journals_ranking.withColumnRenamed('year','publication_year'), on = ['source_id','publication_year'], how = 'left')
                      #.withColumn('inst_id', func.when(func.col('inst_id')=='I118618916',  "I899635006")
                      #                           .when(func.col('inst_id')=="I36085230",  "I899635006")
                      #                           .when(func.col('inst_id')=="I177483745",  "I899635006")
                      #                           .when(func.col('inst_id')=="I4210103002", "I198244214")
                      #                           .when(func.col('inst_id')=="I4210143836", "I198244214")
                      #                           .when(func.col('inst_id')=="I184646667",  "I39804081")
                      #                           .when(func.col('inst_id')=="I102197404",  "I277688954")
                      #                           .when(func.col('inst_id')=='I7171862',  "I2279609970")
                      #                           .when(func.col('inst_id')=="I59807433",  "I2279609970")
                      #                           .when(func.col('inst_id')=="I2800379142", "I4210154111") 
                      #                           .when(func.col('inst_id')=="I3123023596","I56067802")
                      #                           .when(func.col('inst_id')=="I4210095130","I208215962")
                      #                           .otherwise(func.col('inst_id')))
                      .filter(func.col('inst_id').isNotNull())
                      .join(inst.select('inst_id', func.lit(1).alias('in_inst_dataset')), on =['inst_id'], how = 'left')
                      .fillna(0, subset = ['in_inst_dataset'])
                    .withColumn('inst_id', func.when( (func.col('country')=="FR")
                                                     | (func.col('in_inst_dataset') ==1)
                                                     , func.col('inst_id')).otherwise('abroad'))
                      
                    .groupBy('author_id',func.col('publication_year').alias('year'),'work_id')
                    .agg(func.collect_set('inst_id').alias('inst_id_set_y'),
                         *[func.first(col).alias(col) for col in ['citations', 'rank_source_pct']]
                        )
                      .withColumn('coau_set', func.collect_set(func.col('author_id')).over(Window.partitionBy('work_id')))
                    .groupBy('author_id','year')
                    .agg(func.array_compact(func.flatten(func.collect_set('inst_id_set_y'))).alias('inst_id_set_y'),
                         func.array_compact(func.flatten(func.collect_set('coau_set'))).alias('coau_set'),
                         func.countDistinct('work_id').alias('publications'),
                         func.sum('citations').alias('citations'),
                         func.mean('rank_source_pct').alias('avg_rank_source_pct'), 
                         func.sum('rank_source_pct').alias('rank_w_publications'),
                         func.sum((func.col('rank_source_pct')<= 0.5).cast('int')).alias('nr_source_btm_50pct_raw'),
                         func.sum((func.col('rank_source_pct').between(0.5,0.9)).cast('int')).alias('nr_source_mid_40pct_raw'),
                         func.sum((func.col('rank_source_pct')>= 0.8).cast('int')).alias('nr_source_top_20pct_raw'),
                         func.sum((func.col('rank_source_pct')>= 0.9).cast('int')).alias('nr_source_top_10pct_raw'),
                         func.sum((func.col('rank_source_pct')>= 0.95).cast('int')).alias('nr_source_top_5pct_raw'),
                        )

                    #.withColumn('inst_id_set_last_y', func.lag(func.col('inst_id_set_y')).over(Window.partitionBy('author_id').orderBy('year')))
                    #.withColumn('inst_id_receiver', func.explode(func.col('inst_id_set_y')))
                    #.withColumn('inst_id_sender',  func.explode_outer(func.col('inst_id_set_last_y')))
                    #.select('inst_id_receiver','inst_id_sender', 'author_id','year')
                   )

In [8]:
au_inst_level_flow.write.mode('overwrite').parquet('file:\\' + save_path + 'au_inst_level.parquet')

# Add inst-level info

In [7]:
au_inst_level_flow = spark.read.parquet('file:\\' + save_path + 'au_inst_level.parquet')

In [8]:
au_inst_level_flow = (au_inst_level_flow
                    .withColumn('inst_id_set_last_y', func.lag(func.col('inst_id_set_y')).over(Window.partitionBy('author_id').orderBy('year')))
                    .withColumn('inst_id_receiver', func.explode(func.col('inst_id_set_y')))
                    .withColumn('inst_id_sender',  func.explode_outer(func.col('inst_id_set_last_y')))
                    .select('inst_id_receiver','inst_id_sender', 'author_id','year')
                    .distinct()

                     )
au_inst_level_flow.printSchema()

root
 |-- inst_id_receiver: string (nullable = true)
 |-- inst_id_sender: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- year: integer (nullable = true)



In [9]:
au_inst_level_flow.cache()
au_inst_level_flow.show()

+----------------+--------------+-----------+----+
|inst_id_receiver|inst_id_sender|  author_id|year|
+----------------+--------------+-----------+----+
|          abroad|        abroad|A5000000266|2018|
|          abroad|        abroad|A5000000266|2019|
|          abroad|        abroad|A5000002279|2004|
|          abroad|        abroad|A5000002463|2002|
|          abroad|        abroad|A5000005204|1997|
|          abroad|        abroad|A5000005204|1998|
|          abroad|        abroad|A5000005631|2018|
|          abroad|        abroad|A5000005751|1998|
|          abroad|        abroad|A5000005751|2008|
|          abroad|        abroad|A5000005751|2009|
|          abroad|        abroad|A5000005751|2011|
|          abroad|        abroad|A5000005751|2012|
|          abroad|        abroad|A5000005751|2013|
|          abroad|        abroad|A5000005751|2014|
|          abroad|        abroad|A5000005751|2016|
|          abroad|        abroad|A5000005751|2018|
|          abroad|        abroa

In [12]:
au_inst_level_flow.count()

130383692

In [ ]:
au_inst_level_flow.filter( ~ ( 
                                    (func.col('inst_id_sender').isin(['abroad']))
                                    &  (func.col('inst_id_receiver').isin(['abroad']))
                                )).count()

In [10]:
au_inst_level_flow_filtering = (au_inst_level_flow
                                .filter( ~ ( 
                                    (func.col('inst_id_sender').isin(['abroad']))
                                    &  (func.col('inst_id_receiver').isin(['abroad']))
                                ))
                                .join(inst
                                        .select(func.col('inst_id').alias('inst_id_receiver'), 
                                                func.col('parent.parent_id').alias('parent_id_r'))
                                        , on = 'inst_id_receiver', how ='left')
                                .join(inst
                                        .select(func.col('inst_id').alias('inst_id_sender'), 
                                                func.col('parent.parent_id').alias('parent_id_s'))
                                        , on = 'inst_id_sender', how ='left')
            
                                .withColumn('parent_id_r', func.array_except(func.col('parent_id_r'), func.array(func.col('inst_id_receiver'))))
                                .withColumn('parent_id_s', func.array_except(func.col('parent_id_s'), func.array(func.col('inst_id_sender'))))
                                .groupBy('author_id','year')
                                .agg(func.collect_set(func.col('inst_id_receiver')).alias('inst_id_receiver'),
                                     func.collect_set(func.col('inst_id_sender')).alias('inst_id_sender'),
                                     func.array_except(func.transform(func.array_distinct(func.flatten(func.collect_set(func.col('parent_id_r')))), lambda x: func.regexp_replace(x, 'https://openalex.org/', ''))
                                                                                 , func.array(func.lit(None))).alias('all_parent_ids_r'),
                                     
                                     func.array_except(func.transform(func.array_distinct(func.flatten(func.collect_set(func.col('parent_id_s')))), lambda x: func.regexp_replace(x, 'https://openalex.org/', ''))
                                                                                 , func.array(func.lit(None))).alias('all_parent_ids_s')
                               )
                                .withColumn('inst_id_receiver', func.explode(func.array_except(func.col('inst_id_receiver'), func.col('all_parent_ids_r'))))
                                .withColumn('inst_set_sender', func.col('inst_id_sender'))
                                .withColumn('inst_id_sender',   func.explode_outer(func.array_except(func.col('inst_id_sender'), func.col('all_parent_ids_s'))))

                                .select('inst_id_receiver','inst_id_sender', 'inst_set_sender', 'author_id','year')
                               )

In [11]:
au_inst_level_flow_filtering.cache()
au_inst_level_flow_filtering.show()

+----------------+--------------+--------------------+-----------+----+
|inst_id_receiver|inst_id_sender|     inst_set_sender|  author_id|year|
+----------------+--------------+--------------------+-----------+----+
|     I4210098836|          NULL|                  []|A5000016899|1960|
|     I4210152651|   I4210098836|       [I4210098836]|A5000016899|1964|
|     I4210152651|   I4210152651|       [I4210152651]|A5000016899|1965|
|     I4210152651|   I4210152651|[I4210152651, I10...|A5000016899|1966|
|     I3020098449|   I4210152651|[I4210152651, I10...|A5000016899|1966|
|     I4210152651|   I4210152651|[I4210152651, I30...|A5000016899|1967|
|     I4210152651|   I3020098449|[I4210152651, I30...|A5000016899|1967|
|      I100532134|   I4210152651|[I4210152651, I10...|A5000016899|1968|
|     I4210152651|    I100532134|        [I100532134]|A5000016899|1969|
|     I4210152651|   I4210152651|       [I4210152651]|A5000016899|1970|
|     I4210144981|   I4210152651|       [I4210152651]|A500001689

In [12]:
au_inst_level_flow_filtering.count()

20225914

In [25]:
au_inst_level_flow_filtering.filter(func.col('inst_id_receiver')=="I1294671590").count()

2261482

In [26]:
au_inst_level_flow.filter(func.col('inst_id_receiver')=="I1294671590").count()

2863753

In [13]:
au_inst_level_flow.repartition('year')
au_inst_level_flow_filtering.repartition('year')

DataFrame[inst_id_receiver: string, inst_id_sender: string, inst_set_sender: array<string>, author_id: string, year: int]

In [14]:
weights_by_au_year = (au_inst_level_flow_filtering
                      .groupBy('author_id','year')
                      .count()
                      .select('author_id','year', (1/func.col('count')).alias('weights'))
                     )
weights_by_au_year.cache()
weights_by_au_year.show()

+-----------+----+------------------+
|  author_id|year|           weights|
+-----------+----+------------------+
|A5000016899|1960|               1.0|
|A5000016899|1964|               1.0|
|A5000016899|1965|               1.0|
|A5000016899|1966|               0.5|
|A5000016899|1967|               0.5|
|A5000016899|1968|               1.0|
|A5000016899|1969|               1.0|
|A5000016899|1970|               1.0|
|A5000016899|1972|               1.0|
|A5000016899|2009|0.3333333333333333|
|A5000023309|2017|               1.0|
|A5000087584|2018|0.3333333333333333|
|A5000139008|2020|               1.0|
|A5000139201|2011|               0.5|
|A5000139201|2013|              0.25|
|A5000139428|1998|               1.0|
|A5000156980|2000|               1.0|
|A5000156980|2004|               1.0|
|A5000157973|2003|               0.5|
|A5000157973|2005|              0.25|
+-----------+----+------------------+
only showing top 20 rows



In [29]:
au_inst_level_flow.select('author_id').distinct().count()

30301591

In [30]:
au_inst_level_flow_filtering.select('author_id').distinct().count()

1553692

In [13]:
inst.select('secteur').distinct().show()

+---------+
|  secteur|
+---------+
|   public|
|    privé|
|undefined|
+---------+



# author-level info

In [17]:
au_level_info = (au_inst_level_flow
                 #.join(au_to_keep, on= 'author_id', how ='inner')
                 .groupBy('author_id')
                 .agg(func.min('year').alias('entry_year'),
                      func.max('year').alias('exit_year'))
                 .join(au_inst_level_flow
                       .groupBy('author_id', func.col('year').alias("entry_year"))
                       .agg(func.array_sort(func.collect_set(func.col('inst_id_receiver'))).alias('entry_inst_id')
                           )
                       , on = ['author_id', 'entry_year'], how = 'left')
                )

In [18]:
au_level_info.write.mode('overwrite').parquet('file:\\' + save_path + 'au_level_for_inst_dataset.parquet')

In [15]:
au_level_info = spark.read.parquet('file:\\' + save_path + 'au_level_for_inst_dataset.parquet')

In [20]:
au_level_info.count()

30301591

In [16]:
au_inst_level_plus_info = (au_inst_level_flow_filtering
                           .join(au_level_info, on = ['author_id'], how =  'left'))
au_inst_level_plus_info.cache()
au_inst_level_plus_info.count()

20225914

In [22]:
au_inst_level_plus_info.filter(func.col('author_id')=="A5053834730").filter(func.col('inst_id_sender')=="I187986737").show()

+-----------+----------------+--------------+--------------------+----+----------+---------+-------------+
|  author_id|inst_id_receiver|inst_id_sender|     inst_set_sender|year|entry_year|exit_year|entry_inst_id|
+-----------+----------------+--------------+--------------------+----+----------+---------+-------------+
|A5053834730|       I90669466|    I187986737|[I90669466, I1294...|2001|      1972|     2020|[I4210126650]|
|A5053834730|          abroad|    I187986737|[I90669466, I1294...|2001|      1972|     2020|[I4210126650]|
|A5053834730|      I187986737|    I187986737|[I90669466, I1294...|2001|      1972|     2020|[I4210126650]|
|A5053834730|     I1294671590|    I187986737|[I90669466, I1294...|2001|      1972|     2020|[I4210126650]|
|A5053834730|       I90669466|    I187986737|[I90669466, I1879...|2002|      1972|     2020|[I4210126650]|
|A5053834730|          abroad|    I187986737|[I90669466, I1879...|2002|      1972|     2020|[I4210126650]|
|A5053834730|     I1294671590|    I18

# institution-to-institution-level raw flows

In [17]:
pre_inst_flow = (au_inst_level_plus_info
                    .filter(func.col('year').between(1980,2020))
                  .withColumn('stayer', func.array_contains(func.col('inst_set_sender'), func.col('inst_id_receiver')).cast('int'))

                  .withColumn('junior', (func.col('year')-func.col('entry_year')<=5).cast('int'))
                  .withColumn('senior', (func.col('year')-func.col('entry_year')>=15).cast('int'))
                  .withColumn('medium', ((func.col('year')-func.col('entry_year')>5) & (func.col('year')-func.col('entry_year')<15)) .cast('int'))

                  .withColumn('own_entrant_r', (func.array_contains(func.col('entry_inst_id'), func.col('inst_id_receiver')) ).cast('int'))
                  .withColumn('own_entrant_s', (func.array_contains(func.col('entry_inst_id'), func.col('inst_id_sender')) ).cast('int'))

                  .withColumn('foreign_entrant', (func.array_contains(func.col('entry_inst_id'),('abroad')) ).cast('int'))
                 .join(weights_by_au_year, on = ['author_id','year'], how = 'left')
                  )


In [24]:
pre_inst_flow.filter(func.col('inst_id_sender')=="I187986737").show()

+-----------+----+----------------+--------------+--------------------+----------+---------+--------------------+------+------+------+------+-------------+-------------+---------------+-------+
|  author_id|year|inst_id_receiver|inst_id_sender|     inst_set_sender|entry_year|exit_year|       entry_inst_id|stayer|junior|senior|medium|own_entrant_r|own_entrant_s|foreign_entrant|weights|
+-----------+----+----------------+--------------+--------------------+----------+---------+--------------------+------+------+------+------+-------------+-------------+---------------+-------+
|A5000220048|2001|      I187986737|    I187986737|        [I187986737]|      1999|     2009|        [I187986737]|     1|     1|     0|     0|            1|            1|              0|    1.0|
|A5000524924|1999|     I4210141964|    I187986737|[I4210121705, I15...|      1981|     2019|        [I154526488]|     1|     0|     1|     0|            0|            0|              0|    0.2|
|A5000574183|1997|     I421009

In [18]:
pre_inst_flow.write.mode('overwrite').parquet('file:\\' + save_path + 'pre_inst_flow.parquet')

In [31]:
pre_inst_flow = spark.read.parquet('file:\\' + save_path + 'pre_inst_flow.parquet')

In [20]:
pre_inst_flow.count()

19865458

In [ ]:
##### TO CHANGE

In [103]:
inst_level_flows = (pre_inst_flow

                    
                    .groupBy('inst_id_receiver','inst_id_sender', 'year')
                      .agg(func.countDistinct('author_id').alias('total'),
                           func.sum('stayer').alias('stayers'),
                           func.sum(1-func.col('stayer')).alias('movers'),
                        
                           func.sum('weights').alias('total_w'),
                           func.sum(func.col('weights')*func.col('stayer')).alias('stayers_w'),
                           func.sum(func.col('weights')*(1-func.col('stayer'))).alias('movers_w'),


                           *[func.sum(category).alias('total_' + category )
                             for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                           *[func.sum(func.col('stayer')*func.col(category)).alias('stayers_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                           *[func.sum( (1-func.col('stayer'))*func.col(category)).alias('movers_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                        
                           *[func.sum(func.col('weights')*func.col(category)).alias('total_w_' + category)for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],
                           *[func.sum(func.col('weights')*func.col('stayer')*func.col(category)).alias('stayers_w_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],
                           *[func.sum(func.col('weights')*(1-func.col('stayer'))*func.col(category)).alias('movers_w_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],
                           
                           

                          )
                      .join(inst
                            .select(func.col('inst_id').alias('inst_id_receiver'), 
                                  *[func.col(col).alias(col +'_r') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic','idex','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date','merged_inst_id']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_r'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'inst_id_receiver', how ='left')
                      .join(inst
                            .select(func.col('inst_id').alias('inst_id_sender'), 
                                    *[func.col(col).alias(col +'_s') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic','idex','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date','merged_inst_id']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_s'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'inst_id_sender', how ='left')

                    .withColumn('type_r', func.when(func.col('inst_id_receiver')=="abroad", 'abroad').otherwise(func.col('type_r')))
                    .withColumn('type_s', func.when(func.col('inst_id_sender')=="abroad", 'abroad').otherwise(func.col('type_s')))
                    .fillna(0, subset = ['uni_pub_r','uni_pub_s','cnrs_r','cnrs_s','universite_s','universite_r','ecole_r','ecole_s', 'prive_s','prive_r', 'public_s','public_r',
                                        "acces_rce_s",'date_first_idex_s', 'fusion_date_s','merged_inst_id_s', "acces_rce_r", 'date_first_idex_r', 'fusion_date_r','merged_inst_id_r'] )
                    
                    #.withColumn('size_r', func.max(func.col('stayers') * (func.col('inst_id_receiver')==func.col('inst_id_sender')).cast('int'))
                    #
                    #            .over(Window.partitionBy('inst_id_receiver','year')))
                    #
                    #.withColumn('size_s', func.max(func.col('stayers') * (func.col('inst_id_receiver')==func.col('inst_id_sender')).cast('int'))
                    #                  .over(Window.partitionBy('inst_id_sender','year')))
                   )


In [104]:
inst_level_flows.cache()
inst_level_flows.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\socket.py", line 716, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [27]:
inst_level_flows.write.mode('overwrite').parquet('file:\\' + save_path + 'inst_level_flows_unmerged.parquet')

## at the merged institution level

In [32]:
inst_level_flows_merged = (pre_inst_flow
                      .join(inst
                            .select(func.col('inst_id').alias('inst_id_receiver'), 
                                  *[func.col(col).alias(col +'_r') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic','idex','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date','merged_inst_id']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_r'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'inst_id_receiver', how ='left')
                      .join(inst
                            .select(func.col('inst_id').alias('inst_id_sender'), 
                                    *[func.col(col).alias(col +'_s') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic','idex','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date','merged_inst_id']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_s'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'inst_id_sender', how ='left')
                    .withColumn('merged_inst_id_r', func.when(func.col('merged_inst_id_r').isNull(), func.col('inst_id_receiver')).otherwise(func.col('merged_inst_id_r')) )
                    .withColumn('merged_inst_id_s', func.when(func.col('merged_inst_id_s').isNull(), func.col('inst_id_sender')).otherwise(func.col('merged_inst_id_s')) )

                           .withColumn('type_r', func.when(func.col('inst_id_receiver')=="abroad", 'abroad').otherwise(func.col('type_r')))
                    .withColumn('type_s', func.when(func.col('inst_id_sender')=="abroad", 'abroad').otherwise(func.col('type_s')))
                    .fillna(0, subset = ['uni_pub_r','uni_pub_s','cnrs_r','cnrs_s','universite_s','universite_r','ecole_r','ecole_s', 'prive_s','prive_r', 'public_s','public_r',
                                        "acces_rce_s",'date_first_idex_s', 'fusion_date_s','merged_inst_id_s', "acces_rce_r", 'date_first_idex_r', 'fusion_date_r'] )
                    
                    
                    .groupBy('merged_inst_id_r','merged_inst_id_s', 'year')
                      .agg(func.countDistinct('author_id').alias('total'),
                           func.sum('stayer').alias('stayers'),
                           func.sum(1-func.col('stayer')).alias('movers'),
                        
                           func.sum('weights').alias('total_w'),
                           func.sum(func.col('weights')*func.col('stayer')).alias('stayers_w'),
                           func.sum(func.col('weights')*(1-func.col('stayer'))).alias('movers_w'),


                           *[func.sum(category).alias('total_' + category )
                             for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                           *[func.sum(func.col('stayer')*func.col(category)).alias('stayers_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                           *[func.sum( (1-func.col('stayer'))*func.col(category)).alias('movers_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],

                        
                           *[func.sum(func.col('weights')*func.col(category)).alias('total_w_' + category)for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],
                           *[func.sum(func.col('weights')*func.col('stayer')*func.col(category)).alias('stayers_w_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],
                           *[func.sum(func.col('weights')*(1-func.col('stayer'))*func.col(category)).alias('movers_w_' + category) for category in ['junior','senior','medium', 'own_entrant_r','own_entrant_s', 'foreign_entrant']
                           ],


                          *[func.first(col + '_r').alias(col + '_r') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic',
                                                                                 'idex','type_fr','secteur', 'acces_rce','date_first_idex', 
                                                                                 'fusion_date','ecole', 'universite', 'topic_share', 'public', 'prive'] ],
                          *[func.first(col + '_s').alias(col + '_s') for col in ['name','city','parent','uni_pub','cnrs','type','main_topic',
                                                                                 'idex','type_fr','secteur', 'acces_rce','date_first_idex', 
                                                                                 'fusion_date','ecole', 'universite', 'topic_share', 'public', 'prive'] ],
                          )

                    #.withColumn('size_r', func.max(func.col('stayers') * (func.col('inst_id_receiver')==func.col('inst_id_sender')).cast('int'))
                    #
                    #            .over(Window.partitionBy('inst_id_receiver','year')))
                    #
                    #.withColumn('size_s', func.max(func.col('stayers') * (func.col('inst_id_receiver')==func.col('inst_id_sender')).cast('int'))
                    #                  .over(Window.partitionBy('inst_id_sender','year')))
                   )


In [33]:
inst_level_flows_merged.cache()
inst_level_flows_merged.show()

+----------------+----------------+----+-----+-------+------+--------------------+--------------------+-------------------+------------+------------+------------+-------------------+-------------------+---------------------+--------------+--------------+--------------+---------------------+---------------------+-----------------------+-------------+-------------+-------------+--------------------+--------------------+----------------------+-------------------+--------------------+--------------------+---------------------+---------------------+-----------------------+------------------+--------------------+------------------+-----------------------+-----------------------+-------------------------+-------------------+--------------------+--------------------+----------------------+----------------------+------------------------+--------------------+------------+--------------------+---------+------+---------+------------+--------------------+----------+---------+-----------+-----------

In [34]:
all_y_dyads = (inst_level_flows_merged
               .groupBy('merged_inst_id_r','merged_inst_id_s')
               .agg(func.sequence(func.min('year'), func.lit(2020)).alias('year'))
               .withColumn('year', func.explode(func.col('year')))
              )
all_y_dyads.cache()
all_y_dyads.show()

+----------------+----------------+----+
|merged_inst_id_r|merged_inst_id_s|year|
+----------------+----------------+----+
|       I12449238|     I4210104620|2015|
|       I12449238|     I4210104620|2016|
|       I12449238|     I4210104620|2017|
|       I12449238|     I4210104620|2018|
|       I12449238|     I4210104620|2019|
|       I12449238|     I4210104620|2020|
|     I1293226324|          abroad|1980|
|     I1293226324|          abroad|1981|
|     I1293226324|          abroad|1982|
|     I1293226324|          abroad|1983|
|     I1293226324|          abroad|1984|
|     I1293226324|          abroad|1985|
|     I1293226324|          abroad|1986|
|     I1293226324|          abroad|1987|
|     I1293226324|          abroad|1988|
|     I1293226324|          abroad|1989|
|     I1293226324|          abroad|1990|
|     I1293226324|          abroad|1991|
|     I1293226324|          abroad|1992|
|     I1293226324|          abroad|1993|
+----------------+----------------+----+
only showing top

In [35]:
inst_merged_level = (inst
                     .withColumn('merged_inst_id', func.when(func.col('merged_inst_id').isNull(), func.col('inst_id')).otherwise(func.col('merged_inst_id')))
                     .groupBy('merged_inst_id')
                     .agg( *[func.first(func.when(func.col('merged_inst_id')==func.col('inst_id'), func.col(col)), ignorenulls = True).alias(col)
                             for col in inst.columns if col not in ['inst_id','merged_inst_id','acces_rce', 'date_first_idex', 'fusion_date']],
                          *[func.min(func.when((func.col(col) >0) & (func.col(col).isNotNull()),
                                               func.col(col).cast(StringType())).otherwise("0")).alias(col)
                          for col in ['acces_rce','date_first_idex','fusion_date']])
                    )
inst_merged_level.show(truncate = False)

+--------------+---------------------------------------------------+----------+----------------------+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------+-----------------------------------------------------------------------------+-------+----+-----------------------------------+-------------------+----------+---------+---------+---------------+-----------+
|merged_inst_id|name                                               |type      |city                  |homepage_url                                                                   |topic_share                                                                                                                                                     |main_topic                         |parent                             

In [36]:
inst_merged_level.groupBy('merged_inst_id').count().groupBy('count').count().show()

+-----+-----+
|count|count|
+-----+-----+
|    1| 4324|
+-----+-----+



In [43]:
panel_inst_flows_merged = (all_y_dyads
                           .join(inst_level_flows_merged
                                 .select('merged_inst_id_r','merged_inst_id_s','year', *[col for col in  inst_level_flows_merged.columns if 
                                                'total' in col or 'stayer' in col or 'mover' in col])
                                 , on = ['merged_inst_id_r','merged_inst_id_s','year'], how = 'left')
                           .fillna(0, subset = [col for col in  inst_level_flows_merged.columns if 
                                                'total' in col or 'stayer' in col or 'mover' in col])
                           
                           .join(inst_merged_level
                            .select(*[func.col(col).alias(col +'_r') for col in ['merged_inst_id','name','city','uni_pub','cnrs','type','main_topic','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_r'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_r')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'merged_inst_id_r', how ='left')
                      .join(inst_merged_level
                            .select(*[func.col(col).alias(col +'_s') for col in ['merged_inst_id','name','city','uni_pub','cnrs','type','main_topic','type_fr','secteur', 'acces_rce','date_first_idex', 'fusion_date']],
                                   func.concat_ws(';', func.transform(func.col('topic_share'), lambda x: func.to_json(x))).alias('topic_share_s'),
                                   *[ (func.col('type_fr') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                         for type_ in ['Université', 'École']],
                                    *[(func.col('secteur') == type_).cast('int').alias(type_.lower().replace('é', 'e') +'_s')
                                     for type_ in ['public', 'privé']]
                                   )
                            , on = 'merged_inst_id_s', how ='left')

                    .withColumn('type_r', func.when(func.col('merged_inst_id_r')=="abroad", 'abroad').otherwise(func.col('type_r')))
                    .withColumn('type_s', func.when(func.col('merged_inst_id_s')=="abroad", 'abroad').otherwise(func.col('type_s')))
                    .fillna(0, subset = ['uni_pub_r','uni_pub_s','cnrs_r','cnrs_s','universite_s','universite_r','ecole_r','ecole_s', 'prive_s','prive_r', 'public_s','public_r',
                                        "acces_rce_s",'date_first_idex_s', 'fusion_date_s','merged_inst_id_s', "acces_rce_r", 'date_first_idex_r', 'fusion_date_r'] )
                    
                    
                    #.withColumn('size_r', func.max(func.col('stayers') * (func.col('merged_inst_id_r')==func.col('merged_inst_id_s')).cast('int'))
                    #
                    #            .over(Window.partitionBy('merged_inst_id_r','year')))
                    #
                    #.withColumn('size_s', func.max(func.col('stayers') * (func.col('merged_inst_id_r')==func.col('merged_inst_id_s')).cast('int'))
                    #                  .over(Window.partitionBy('merged_inst_id_s','year')))

                          )


In [44]:
panel_inst_flows_merged.write.mode('overwrite').parquet('file:\\' + save_path + 'inst_level_flows.parquet')

In [45]:
panel_inst_flows_merged = spark.read.parquet('file:\\' + save_path + 'inst_level_flows.parquet')

In [46]:
panel_inst_flows_merged.count()

6501680

In [47]:
panel_inst_flows_merged.select('merged_inst_id_s','merged_inst_id_r').distinct().count()

451519

In [48]:
panel_inst_flows_merged.groupBy('merged_inst_id_s','merged_inst_id_r','year').count().groupBy('count').count().show()

+-----+-------+
|count|  count|
+-----+-------+
|    1|6501680|
+-----+-------+



In [49]:
panel_inst_flows_merged.select('merged_inst_id_s','merged_inst_id_r').filter(func.col('merged_inst_id_s') != func.col('merged_inst_id_r')).distinct().count()

445741

In [50]:
panel_inst_flows_merged.filter(func.col('movers_w')>0).select('merged_inst_id_s','merged_inst_id_r').filter(func.col('merged_inst_id_s') != func.col('merged_inst_id_r')).distinct().count()

391327

## Merged parent level

In [52]:
spark.catalog.clearCache()
spark.stop()

# Institution level

In [14]:
inst_level_flows = (inst_level_flows
                    .groupBy(func.col('inst_id_receiver').alias('inst_id'), 'year')
                    .agg(*[func.first(col + '_r').alias(col)
                           for col in ['name','city','parent','fused','uni_pub','cnrs','type','main_topic','idex','type_fr','secteur','prive','ecole','universite','public']],
                         *[func.sum(col).alias(col) for col in inst_level_flows.columns if "stayer" in col or "mover" in col or "total" in col]
                        )
                   )

In [15]:
inst_level_flows.show()

+----------+----+--------------------+--------------------+--------------------+-----+-------+----+----------+--------------------+--------------------+----------+---------+-----+-----+----------+------+-----+-------+------+------------------+------------------+------------------+------------+------------+------------+-------------------+-------------------+---------------------+--------------+--------------+--------------+---------------------+---------------------+-----------------------+-------------+-------------+-------------+--------------------+--------------------+----------------------+------------------+------------------+------------------+---------------------+---------------------+-----------------------+------------------+-------------------+------------------+-----------------------+-----------------------+-------------------------+------------------+------------------+------------------+----------------------+----------------------+------------------------+
|   inst_id|

In [16]:
inst_level_flows.write.mode('overwrite').parquet('file:\\' + save_path + 'unilateral_inst_level_flows.parquet')

In [17]:
spark.catalog.clearCache()
spark.stop()

In [ ]:
inst_level_flows_moves = (inst_level_flows

                          .drop('stayers')
                         )
inst_level_flows_moves.show()

In [50]:
au_inst_level_flow.filter( (func.col('inst_id_receiver')=="I57995698")  & (func.col('inst_id_sender').isin(["I187986737"]))
                          & (func.col('year')==2020)).select('author_id','inst_id_sender','inst_id_receiver').count()

3

In [55]:
au_inst_level_flow.filter( (func.col('inst_id_receiver')=="I57995698")  & (func.col('inst_id_sender')=="I187986737")
                          & (func.col('year')==2020)).select('author_id').distinct().count()

3

In [54]:
inst_level_flows.filter( (func.col('inst_id_receiver')=="I57995698")  & (func.col('inst_id_sender')=="I187986737")
                        ).select('year','inst_id_sender','inst_id_receiver','stayers','movers','movers_w').show()

+----+--------------+----------------+-------+------+--------+
|year|inst_id_sender|inst_id_receiver|stayers|movers|movers_w|
+----+--------------+----------------+-------+------+--------+
|2015|    I187986737|       I57995698|      1|     0|     0.0|
|2018|    I187986737|       I57995698|      3|     0|     0.0|
|2006|    I187986737|       I57995698|      1|     0|     0.0|
|2001|    I187986737|       I57995698|      1|     0|     0.0|
|2005|    I187986737|       I57995698|      1|     0|     0.0|
|2010|    I187986737|       I57995698|      2|     0|     0.0|
|2017|    I187986737|       I57995698|      2|     0|     0.0|
|2016|    I187986737|       I57995698|      2|     1|     0.5|
|2011|    I187986737|       I57995698|      2|     1|    0.25|
|2008|    I187986737|       I57995698|      2|     0|     0.0|
|1994|    I187986737|       I57995698|      1|     0|     0.0|
|2012|    I187986737|       I57995698|      2|     0|     0.0|
|2002|    I187986737|       I57995698|      1|     0|  

In [55]:
inst_level_flows.filter(func.col('inst_id_receiver')=="I57995698").groupBy('year').agg(func.sum('movers').alias('movers'), func.sum('stayers').alias('stayers'),
                                                                                             func.sum('movers_w').alias('movers_w'),
                                                                                      func.sum('stayers_w').alias('stayers_w')).sort('year').collect()

[Row(year=1980, movers=1, stayers=0, movers_w=1.0, stayers_w=0.0),
 Row(year=1982, movers=3, stayers=0, movers_w=1.5, stayers_w=0.0),
 Row(year=1984, movers=2, stayers=0, movers_w=0.3333333333333333, stayers_w=0.0),
 Row(year=1985, movers=1, stayers=0, movers_w=1.0, stayers_w=0.0),
 Row(year=1988, movers=5, stayers=2, movers_w=1.5833333333333333, stayers_w=0.5),
 Row(year=1989, movers=1, stayers=0, movers_w=0.3333333333333333, stayers_w=0.0),
 Row(year=1990, movers=1, stayers=0, movers_w=0.5, stayers_w=0.0),
 Row(year=1991, movers=1, stayers=0, movers_w=0.3333333333333333, stayers_w=0.0),
 Row(year=1992, movers=3, stayers=0, movers_w=1.0, stayers_w=0.0),
 Row(year=1993, movers=0, stayers=5, movers_w=0.0, stayers_w=0.8333333333333333),
 Row(year=1994, movers=1, stayers=3, movers_w=0.25, stayers_w=0.3333333333333333),
 Row(year=1995, movers=1, stayers=0, movers_w=1.0, stayers_w=0.0),
 Row(year=1996, movers=11, stayers=0, movers_w=5.083333333333333, stayers_w=0.0),
 Row(year=1997, movers=

In [16]:
inst_level_flows.filter(func.col('inst_id_receiver')=="I57995698").filter(func.col('year')=="2020").show()

+----------------+--------------+----+-----+-------+------+--------------------+--------------------+--------------------+------------+------------+------------+-------------------+-------------------+---------------------+--------------+--------------+--------------+---------------------+---------------------+-----------------------+-------------+-------------+-------------+--------------------+--------------------+----------------------+--------------+--------------------+-------------------+---------------------+---------------------+-----------------------+----------------+--------------------+-------------------+-----------------------+-----------------------+-------------------------+---------------+--------------------+-------------------+----------------------+----------------------+------------------------+--------------------+------+--------------------+-------+---------+------+---------+--------------------+--------------------+-------+---------+---------+-------------------

In [54]:
(inst_level_flows_moves
 .filter(func.col('inst_id_receiver')=="x")
 .filter(func.col('year')==2013)
 .withColumn('name_s', func.when(func.col('name_s').isNull(), func.col('inst_id_sender')).otherwise(func.col('name_s')))
 .select('name_r','name_s','movers','movers_w').filter('movers>0')).show(100,truncate = False)

+-------------------------+--------------------------------------------------------------------------+------+--------------------+
|name_r                   |name_s                                                                    |movers|movers_w            |
+-------------------------+--------------------------------------------------------------------------+------+--------------------+
|Paris School of Economics|I4210103002                                                               |1     |0.04                |
|Paris School of Economics|Alimentation et Sciences Sociales                                         |1     |0.5                 |
|Paris School of Economics|Aix-Marseille Université                                                  |1     |0.0625              |
|Paris School of Economics|Institut national d'études démographiques                                 |1     |0.5                 |
|Paris School of Economics|Laboratoire Psychologie de la Perception                

In [ ]:
to_plot = (inst_level_flows_moves
           .filter( (func.col('inst_id_receiver') != func.col('inst_id_sender')))
           .filter(func.col('inst_id_sender')!="abroad")
           .groupBy('year','uni_pub_r')
           .agg(func.sum('movers').alias('total_flow'),
                func.mean('movers').alias('avg_flow'),
                func.sum('total').alias('total'),
                func.sum('movers_w').alias('total_flow_w'),
                func.mean('movers_w').alias('avg_flow_w'),
                func.sum('total_w').alias('total_w')
               )
          ).toPandas()

In [ ]:
year_of_change = [2007]
graph_3(to_plot[to_plot['year']>=1997], 'year','total_flow_w', 'uni_pub_r', [0,1])

In [ ]:
graph_3(to_plot[to_plot['year']>=1997], 'year','avg_flow', 'uni_pub_r', [0,1])

In [ ]:
graph_3(to_plot[to_plot['year']>=1997], 'year','total_w', 'uni_pub_r', [0,1])

In [ ]:
to_plot = (inst_level_flows_moves
           .filter( (func.col('inst_id_receiver') != func.col('inst_id_sender')))
           .filter(func.col('inst_id_receiver')!="abroad")
           .filter(func.col('type_r')!="company")
           .groupBy('year','uni_pub_s')
           .agg(func.sum('movers').alias('total_flow'),
                func.mean('movers').alias('avg_flow'),
                func.sum('total').alias('total'),
                func.sum('movers_w').alias('total_flow_w'),
                func.mean('movers_w').alias('avg_flow_w'),
                func.sum('total_w').alias('total_w')
               )
          ).toPandas()
graph_3(to_plot[to_plot['year']>=1997], 'year','total_flow_w', 'uni_pub_s', [0,1])
graph_3(to_plot[to_plot['year']>=1997], 'year','avg_flow_w', 'uni_pub_s', [0,1])

# Lab-level output

In [30]:
au_inst_level_flow_filtering.show()

+----------------+--------------+--------------------+-----------+----+
|inst_id_receiver|inst_id_sender|     inst_set_sender|  author_id|year|
+----------------+--------------+--------------------+-----------+----+
|     I4210098836|          NULL|                  []|A5000016899|1960|
|     I4210152651|   I4210098836|       [I4210098836]|A5000016899|1964|
|     I4210152651|   I4210152651|       [I4210152651]|A5000016899|1965|
|     I4210152651|   I4210152651|[I4210152651, I10...|A5000016899|1966|
|     I3020098449|   I4210152651|[I4210152651, I10...|A5000016899|1966|
|     I4210152651|   I4210152651|[I4210152651, I30...|A5000016899|1967|
|     I4210152651|   I3020098449|[I4210152651, I30...|A5000016899|1967|
|      I100532134|   I4210152651|[I4210152651, I10...|A5000016899|1968|
|     I4210152651|    I100532134|        [I100532134]|A5000016899|1969|
|     I4210152651|   I4210152651|       [I4210152651]|A5000016899|1970|
|     I4210144981|   I4210152651|       [I4210152651]|A500001689

In [31]:
w_for_lags = Window.partitionBy('inst_id','author_id').orderBy('year')

au_inst_y = (au_inst_level_plus_info
               .select('author_id','year', func.col('inst_id_receiver').alias('inst_id'), 'inst_set_sender', 'entry_year', 'exit_year','entry_inst_id')
             .withColumn('stayer', (func.array_contains(func.col('inst_set_sender'), func.col('inst_id'))  ).cast('int'))
             .withColumn('first_y_inst', func.min('year').over(Window.partitionBy('author_id','inst_id')))
             .withColumn('past_5_y_entrant', func.greatest(
                        1-func.col('stayer'), (func.col('year')-func.col('first_y_inst') <=5).cast('int')  ))
             .withColumn('foreign_entrant', (func.array_contains(func.col('entry_inst_id'),('abroad')) ).cast('int'))
             .withColumn('from_abroad', func.max((func.col('inst_set_sender')==func.array([func.lit('abroad')])).cast('int')).over(Window.partitionBy('author_id','inst_id')))
             .withColumn('new_foreign_entrant', func.col('foreign_entrant')*func.col('past_5_y_entrant'))
             .withColumn('new_from_abroad', func.col('from_abroad')*func.col('past_5_y_entrant'))
            )
au_inst_y.cache()
au_inst_y.show()

+-----------+----+-----------+--------------------+----------+---------+--------------------+------+------------+----------------+---------------+-----------+-------------------+---------------+
|  author_id|year|    inst_id|     inst_set_sender|entry_year|exit_year|       entry_inst_id|stayer|first_y_inst|past_5_y_entrant|foreign_entrant|from_abroad|new_foreign_entrant|new_from_abroad|
+-----------+----+-----------+--------------------+----------+---------+--------------------+------+------------+----------------+---------------+-----------+-------------------+---------------+
|A5000016899|1968| I100532134|[I4210152651, I10...|      1960|     2009|       [I4210098836]|     1|        1968|               1|              0|          0|                  0|              0|
|A5000016899|1966|I3020098449|[I4210152651, I10...|      1960|     2009|       [I4210098836]|     0|        1966|               1|              0|          0|                  0|              0|
|A5000016899|2009|I421008

In [32]:
au_inst_y.write.mode('overwrite').parquet('file:\\' + int_save_path + 'au_inst_y_step_3.parquet')

In [7]:
au_inst_y = spark.read.parquet('file:\\' + int_save_path + 'au_inst_y_step_3.parquet')

In [34]:
au_to_select = au_inst_y.select('author_id').distinct()
au_to_select.count()

1553692

In [35]:
au_inst_y.count()

20225914

In [8]:
inst_y = (au_inst_y
          .groupBy('inst_id','year')
          .agg(        
               func.array_compact(
                        func.collect_set(func.when(func.col('past_5_y_entrant')==1, func.col('author_id')))).alias('all_5_y'),
               func.array_compact(
                        func.collect_set(func.when(func.col('new_from_abroad')==1, func.col('author_id')))).alias('all_5_y_abroad'),
               func.array_compact(
                        func.collect_set(func.when(func.col('new_foreign_entrant')==1, func.col('author_id')))).alias('all_5_y_foreign_entrants'),

              func.collect_set(func.col('author_id')).alias('all_authors_y'),
               func.count('author_id').alias('total')
              )
         )
inst_y.cache()

DataFrame[inst_id: string, year: int, all_5_y: array<string>, all_5_y_abroad: array<string>, all_5_y_foreign_entrants: array<string>, all_authors_y: array<string>, total: bigint]

In [9]:
inst_y.count() #94641

109948

In [10]:
inst_y.show()

+-----------+----+--------------------+--------------------+------------------------+--------------------+-----+
|    inst_id|year|             all_5_y|      all_5_y_abroad|all_5_y_foreign_entrants|       all_authors_y|total|
+-----------+----+--------------------+--------------------+------------------------+--------------------+-----+
| I102197404|1980|[A5013835025, A50...|[A5005660288, A50...|    [A5078501285, A50...|[A5013835025, A50...| 1302|
| I106530797|1996|[A5014014363, A50...|                  []|           [A5062050087]|[A5014014363, A50...|    3|
| I110736937|1974|[A5088255468, A50...|       [A5082604273]|           [A5082604273]|[A5088255468, A50...|    8|
| I112936343|2006|[A5088701076, A50...|[A5004952249, A50...|    [A5004952249, A50...|[A5063958148, A50...|  312|
| I117345614|1995|[A5065129513, A50...|       [A5006144861]|    [A5065129513, A50...|[A5065129513, A50...|    3|
|  I11935315|2020|[A5082738725, A50...|[A5082738725, A51...|    [A5082738725, A51...|[A508273872

In [39]:
inst_y.withColumn('author_id', func.explode("all_authors_y")).count()

9812619

In [40]:
works_subset = (works_au_af.select('author_id','work_id')
                      .join(au_to_select, on = 'author_id')  
                .select('work_id').distinct()
                     )
works_subset.write.mode('overwrite').parquet('file:\\' + int_save_path + 'works_subset_for_labs.parquet')

In [41]:
works_subset= spark.read.parquet('file:\\' + int_save_path + 'works_subset_for_labs.parquet')

In [44]:
works_au_af_subset = (works_au_af
                      .select('work_id','publication_year','author_id','source_id','citations','primary_topic')
                      .join(works_subset, on = 'work_id')                      
                      .groupBy('work_id')
                      .agg(func.first('publication_year').alias('year'),
                           *[func.first(col).alias(col) for col in ['source_id','citations','primary_topic']],
                           func.collect_set('author_id').alias('all_author_id'))
                      #.withColumn('author_id', func.explode(func.col('all_author_id')))
                     )

In [45]:
works_au_af_subset.write.mode('overwrite').parquet('file:\\' + int_save_path + 'works_subset_for_labs_w_info.parquet')

In [11]:
works_au_af_subset = spark.read.parquet('file:\\' + int_save_path + 'works_subset_for_labs_w_info.parquet')

In [13]:
au_coau_y = (works_au_af_subset
                      .withColumn('author_id', func.explode("all_author_id"))
                      .groupBy('author_id','year')
                      .agg(func.concat_ws(',', func.array_compact(func.flatten(func.collect_set('all_author_id')))).alias('coau_set'))
            )
au_coau_y.write.mode('overwrite').parquet('file:\\' + int_save_path + 'au_coau_y_for_labs.parquet')

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\socket.py", line 716, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o149.parquet

In [ ]:
au_coau_y= spark.read.parquet('file:\\' + int_save_path + 'au_coau_y_for_labs.parquet')
au_coau_y.show()

In [ ]:
au_coau_y.count()

In [ ]:
inst_work_au = (inst_y
                .withColumn('author_id', func.explode("all_authors_y"))
                .join(works_au_af_subset
                      .withColumn('author_id', func.explode("all_author_id"))
                      .drop('all_author_id'),
                      on  = ['author_id','year'], how = 'left')
                .join(au_coau_y,
                      on = ['author_id','year'], how = 'left')
               )
inst_work_au.write.mode('overwrite').parquet('file:\\' + int_save_path + 'inst_work_au.parquet')

In [17]:
inst_work_au.cache()
inst_work_au.show()

Py4JJavaError: An error occurred while calling o161.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 27.0 failed 1 times, most recent failure: Lost task 7.0 in stage 27.0 (TID 620) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$ano

In [35]:
inst_work_level = (inst_work_au
                    .join(journals_ranking, on = ['source_id','year'], how = 'left')
                    .groupBy('inst_id','work_id','year')
                    .agg(*[func.first(col).alias(col) for col in ['citations', 'rank_source_pct']],
                         *[func.max( (func.array_contains(func.col(cat_col),func.col('author_id'))).cast('int') ).alias('is_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']],
                         *[func.max( ((func.arrays_overlap(func.col('coau_set'), func.col(cat_col)))
                                    & (~func.array_contains(func.col(cat_col),func.col('author_id'))) ).cast('int')
                                   ).alias('is_coau_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']],
                         *[func.max( ((~func.arrays_overlap(func.col('coau_set'), func.col(cat_col)))
                                    & (~func.array_contains(func.col(cat_col),func.col('author_id'))) ).cast('int')
                                   ).alias('is_no_coau_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']]
                        )

                  )
inst_work_level.cache()
inst_work_level.show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\pyspark\errors\exceptions\captured.py", line 179, in deco
    return f(*a, **kw)
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <exception str() failed>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\anaconda3\envs\pyspark_env\lib\socket.py", line 716, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant

During handling of the above exception, another exception occurred:

Traceba

Py4JError: py4j does not exist in the JVM

In [18]:
au_y_coau = (au_inst_level_plus_info
             .select('author_id').distinct()
             .join(works_au_af.filter(func.col('publication_year').between(1980,2020))
                      .withColumn('coau_set', func.collect_set(func.col('author_id')).over(Window.partitionBy('publication_year')))
                   .select('coau_set','author_id', func.col('publication_year').alias('year'))
                   , on = 'author_id', how= 'left')
            )
au_y_coau.cache()
au_y_coau.show()

Py4JJavaError: An error occurred while calling o382.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 36 in stage 57.0 failed 1 times, most recent failure: Lost task 36.0 in stage 57.0 (TID 2903) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeUnalignedBytes(UnsafeWriter.java:127)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:110)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.window.WindowExec$$anon$1.next(WindowExec.scala:198)
	at org.apache.spark.sql.execution.window.WindowExec$$anon$1.next(Wind

In [22]:
works_subset = (works_au_af
                   .filter(func.col('publication_year').between(1980,2020))
                     .select('author_id','work_id','publication_year','citations','source_id')
                .join(au_inst_level_plus_info.select('author_id').distinct(), on ='author_id')
               )
works_subset.cache()
works_subset.show()

Py4JJavaError: An error occurred while calling o437.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 131 in stage 66.0 failed 1 times, most recent failure: Lost task 131.0 in stage 66.0 (TID 2731) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler

In [26]:

inst_work_level = (works_subset
                      .withColumnRenamed('publication_year','year')
                      .join(journals_ranking, on = ['source_id','year'], how = 'left')
                      .join(inst_y
                            .withColumn('author_id', func.explode("all_authors_y")), on = ['author_id','year'], how = 'inner')
                    .groupBy('inst_id','work_id','year')
                    .agg(*[func.first(col).alias(col) for col in ['citations', 'rank_source_pct']],
                         *[func.max( (func.array_contains(func.col(cat_col),func.col('author_id'))).cast('int') ).alias('is_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']],
                         *[func.max( ((func.arrays_overlap(func.col('coau_set'), func.col(cat_col)))
                                    & (~func.array_contains(func.col(cat_col),func.col('author_id'))) ).cast('int')
                                   ).alias('is_coau_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']],
                         *[func.max( ((~func.arrays_overlap(func.col('coau_set'), func.col(cat_col)))
                                    & (~func.array_contains(func.col(cat_col),func.col('author_id'))) ).cast('int')
                                   ).alias('is_no_coau_' + cat_col)
                           for cat_col in ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants']]
                        )

                  )
inst_work_level.cache()
inst_work_level.show()

Py4JJavaError: An error occurred while calling o545.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 98.0 failed 1 times, most recent failure: Lost task 5.0 in stage 98.0 (TID 3757) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.unsafe.types.UTF8String.fromAddress(UTF8String.java:132)
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.getUTF8String(UnsafeRow.java:382)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec$.$anonfun$prepareShuffleDependency$6(ShuffleExchangeExec.scala:323)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec$.$anonfun$prepareShuffleDependency$6$adapted(ShuffleExchangeExec.scala:323)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec$$$Lambda$3892/0x0000000801818840.apply(Unknown Sour

In [89]:
from itertools import product
inst_level_output = (inst_work_level
                    .groupBy('inst_id','year')
                    .agg(func.countDistinct('work_id').alias('publications'),
                         func.sum('citations').alias('citations'),
                         func.mean('rank_source_pct').alias('avg_rank_source_pct'), 
                         func.sum('rank_source_pct').alias('rank_w_publications'),
                         func.sum((func.col('rank_source_pct')<= 0.5).cast('int')).alias('nr_source_btm_50pct'),
                         func.sum((func.col('rank_source_pct').between(0.5,0.9)).cast('int')).alias('nr_source_mid_40pct'),
                         func.sum((func.col('rank_source_pct')>= 0.8).cast('int')).alias('nr_source_top_20pct'),
                         func.sum((func.col('rank_source_pct')>= 0.9).cast('int')).alias('nr_source_top_10pct'),
                         func.sum((func.col('rank_source_pct')>= 0.95).cast('int')).alias('nr_source_top_5pct'),
                         func.sum((func.col('rank_source_pct')>= 0.99).cast('int')).alias('nr_source_top_1pct'),

                         *[ func.sum(cat_coau + cat_new).alias(cat_coau.replace('is','publications')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*func.col('citations') ).alias(cat_coau.replace('is','citations')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*func.col('rank_source_pct') ).alias(cat_coau.replace('is','rank_w_pub')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.mean(func.col(cat_coau + cat_new)*func.col('rank_source_pct') ).alias(cat_coau.replace('is','avg_rank')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct')<= 0.5).cast('int') ).alias(cat_coau.replace('is','nr_source_btm_50pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct').between(0.5,0.9)).cast('int')).alias(cat_coau.replace('is','nr_source_mid_40pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct')>= 0.8).cast('int') ).alias(cat_coau.replace('is','nr_source_top_20pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct')>= 0.9).cast('int') ).alias(cat_coau.replace('is','nr_source_top_10pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct')>= 0.95).cast('int') ).alias(cat_coau.replace('is','nr_source_top_5pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],
                         *[ func.sum(func.col(cat_coau + cat_new)*(func.col('rank_source_pct')>= 0.99).cast('int') ).alias(cat_coau.replace('is','nr_source_top_1pct')+cat_new) 
                           for cat_coau, cat_new in product(['is_','is_coau_','is_no_coau_'], ['all_5_y','all_5_y_abroad', 'all_5_y_foreign_entrants'])],


                        )
                   )
inst_level_output.cache()
inst_level_output.show()

Py4JJavaError: An error occurred while calling o5082.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 585 in stage 431.0 failed 1 times, most recent failure: Lost task 585.0 in stage 431.0 (TID 11793) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.parquet.bytes.HeapByteBufferAllocator.allocate(HeapByteBufferAllocator.java:32)
	at org.apache.parquet.hadoop.ParquetFileReader$ConsecutivePartList.readAll(ParquetFileReader.java:1846)
	at org.apache.parquet.hadoop.ParquetFileReader.internalReadRowGroup(ParquetFileReader.java:990)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextRowGroup(ParquetFileReader.java:940)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextFilteredRowGroup(ParquetFileReader.java:1100)
	at org.apache.spark.sql.exe

In [12]:
inst_level_output.write.mode('overwrite').parquet('file:\\' + save_path + 'lab_level_output.parquet')

Py4JJavaError: An error occurred while calling o1226.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 43.0 failed 1 times, most recent failure: Lost task 13.0 in stage 43.0 (TID 2765) (CDF-9FB1674.priv.college-de-france.fr executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$an